In [30]:
import os
import re
import sys
import tensorflow as tf
import tensorflow.python.platform
from tensorflow.python.platform import gfile
import numpy as np
import pandas as pd
import sklearn
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC, LinearSVC
import matplotlib.pyplot as plt
import pickle
import av

In [31]:
path_to_video = 'test.mp4'
path_to_frames = 'frames/'
images_dir = path_to_frames
model_dir = '../imagenet/classify_image_graph_def.pb'
results_dir = '../results/'
features_file = os.path.join(results_dir, 'features')
saliency_matrix_file = os.path.join(results_dir, 'saliency_matrix')

In [32]:
def frame_extractor(path_to_video, path_to_frames):
    container = av.open(path_to_video)
    video = next(s for s in container.streams if s.type == b'video')
    for packet in container.demux(video):
        for frame in packet.decode():
            frame.to_image().save(path_to_frames + '/frame-%06d.jpg' % frame.index)

In [33]:
class FeatureProcessor:
    def __init__(self, model_path, images_dir, sampling_rate=10, saliency_threshold=0.8):
        self.model_path = model_path
        self.images_path = sorted([images_dir + f for f in os.listdir(images_dir) if re.search('jpg|JPG', f)])
        self.sampling_rate = sampling_rate
        self.saliency_threshold = saliency_threshold
        
    def create_graph(self):
        with gfile.FastGFile(self.model_path, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(graph_def, name='')
    
    def get_layer_list(self):
        self.create_graph()
        with tf.Session() as sess:
            operation_list = sess.graph.get_operations()
            for op in operation_list:
                print op.name
                print op.values().get_shape()

    # TODO : Experiment with other layers.            
    def extract_features(self, layer_name='pool_3:0', nb_features=2048):
        features = np.empty((len(self.images_path),nb_features))
        labels = []
        self.create_graph()
        with tf.Session() as sess:
            next_to_last_tensor = sess.graph.get_tensor_by_name(layer_name)
            for index, image in enumerate(self.images_path):
                if (index % self.sampling_rate == 0):
                    print('Processing %s...' % (image))
                    if not gfile.Exists(image):
                        tf.logging.fatal('File does not exist %s', image)
                        continue
                    image_data = gfile.FastGFile(image, 'rb').read()
                    predictions = sess.run(next_to_last_tensor, {'DecodeJpeg/contents:0': image_data})
                    features[index, :] = np.squeeze(predictions)
        return features
    
    def compute_change_points(self, features):
        features = preprocessing.normalize(features, norm='l2')
        features_1 = np.vstack((np.zeros((1, features.shape[1])), features))
        features_2 = np.vstack((features,np.zeros((1, features.shape[1]))))
        saliency_matrix = features_2 - features_1
        
        distance = []
        for vector in saliency_matrix:
            distance.append(np.linalg.norm(vector))
        print distance
        distance = np.asarray(distance)
        
        change_points = (np.where(distance > self.saliency_threshold)) * self.sampling_rate
        return change_points

In [35]:
if __name__ == "__main__":
    frame_extractor(path_to_video, path_to_frames)
    feature_processor = FeatureProcessor(model_dir, images_dir)
    feature_processor.create_graph()
    features = feature_processor.extract_features()
    change_points = feature_processor.compute_change_points(features)

Processing frames/frame-000000.jpg...
Processing frames/frame-000010.jpg...
Processing frames/frame-000020.jpg...
Processing frames/frame-000030.jpg...
Processing frames/frame-000040.jpg...
Processing frames/frame-000050.jpg...
Processing frames/frame-000060.jpg...
Processing frames/frame-000070.jpg...
Processing frames/frame-000080.jpg...
Processing frames/frame-000090.jpg...
Processing frames/frame-000100.jpg...
Processing frames/frame-000110.jpg...
Processing frames/frame-000120.jpg...
Processing frames/frame-000130.jpg...
Processing frames/frame-000140.jpg...
Processing frames/frame-000150.jpg...
Processing frames/frame-000160.jpg...
Processing frames/frame-000170.jpg...
Processing frames/frame-000180.jpg...
Processing frames/frame-000190.jpg...
Processing frames/frame-000200.jpg...
Processing frames/frame-000210.jpg...
Processing frames/frame-000220.jpg...
Processing frames/frame-000230.jpg...
Processing frames/frame-000240.jpg...
Processing frames/frame-000250.jpg...
Processing f